In [1]:
import pandas as pd
import os
import numpy as np
import string

In [2]:
def search_homes(filename, date, keyword):
    """
    Method that searches through the Data_dump dict structure
    
    Returns:
        vals: dict where keys include keyword and values are counts
        home_count: # of unique homes where the keyword is present
    """
    lennar = pd.read_json(filename)
    clean = lennar.drop_duplicates(subset=['Availability', 'Price', 'Beds', 'Baths', 'Sqft', 'Address', 'Community', 'URL'],
                    keep='first',
                    ignore_index=True)
    
    keyword = keyword.lower()
    vals = dict()
    home_count = 0
    index = 0
    total_features = 0
    
    for d in clean.Data_Dump:
        index += 1
        lines = [item.lower() for sublist in d.values() for item in sublist]
        total_features += len(lines)
    
        # get rid of characters we don't want
        cleaned_lines = [l.strip().replace('\n', '').replace('\r', '').replace('®', '').
                         replace('™','').replace('¹', '').replace('”','') for l in lines]

        # remove punc other than dash
        ultra_cleaned_lines = [cl.translate(str.maketrans('', '', string.punctuation.replace('-',''))) 
                               for cl in cleaned_lines]

        cleaned_words = [i for l in ultra_cleaned_lines for i in l.split()]
        
        if keyword in cleaned_words:
            home_count += 1
    
    return home_count, total_features

In [18]:
def market_analysis(date, keyword):
    json_files = []
    path = f'./data-{date}/'
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith('.json'):
                json_files.append(root+'/'+name)
                
    df = pd.DataFrame([], columns = ['county', 'state', f'n_homes_{date}', 
                                     f'{keyword}_homes_{date}', f'total_features_{date}'])
    for jf in json_files:
        tmp = pd.read_json(jf)
        state = jf.split('/')[2]
        county = jf.split('/')[3].split('-final')[0]
        n_homes, n_features = search_homes(jf, date, keyword)
        
        if '.ipynb_checkpoints' in county:
            continue

        df.loc[len(df)] = [county, state, tmp.shape[0], n_homes, n_features]
    
    return df

In [19]:
df1 = market_analysis('06-2023', 'usb')
df1['features_per_home_06-2023'] = df1['total_features_06-2023'] / df1['n_homes_06-2023']

In [20]:
df1.shape

(70, 6)

In [21]:
df2 = market_analysis('10-2022', 'usb')
df2['features_per_home_10-2022'] = df2['total_features_10-2022'] / df2['n_homes_10-2022']

In [22]:
df2.shape

(67, 6)

In [23]:
counties = df1.merge(df2, on=['county', 'state'], how='outer')

In [24]:
counties

,county,state,n_homes_06-2023,usb_homes_06-2023,total_features_06-2023,features_per_home_06-2023,n_homes_10-2022,usb_homes_10-2022,total_features_10-2022,features_per_home_10-2022
0,raleigh,north-carolina,177.0,0.0,10884.0,61.491525,319.0,0.0,23188.0,72.689655
1,wilmington,north-carolina,110.0,0.0,5472.0,49.745455,103.0,0.0,5917.0,57.446602
2,charlotte,north-carolina,242.0,232.0,20441.0,84.466942,199.0,171.0,18411.0,92.517588
3,greensboro,north-carolina,3.0,0.0,231.0,77.000000,NaN,NaN,NaN,NaN
4,chicago,illinois,185.0,0.0,13158.0,71.124324,192.0,0.0,12808.0,66.708333
...,...,...,...,...,...,...,...,...,...,...
73,inland-northwest,washington,NaN,NaN,NaN,NaN,23.0,0.0,2429.0,105.608696
74,seattle,washington,NaN,NaN,NaN,NaN,175.0,0.0,21607.0,123.468571
75,jefferson-county,west-virginia,NaN,NaN,NaN,NaN,24.0,0.0,310.0,12.916667
76,madison,wisconsin,NaN,NaN,NaN,NaN,27.0,0.0,88.0,3.259259


In [25]:
states = counties.groupby('state').sum()

In [26]:
states

,n_homes_06-2023,usb_homes_06-2023,total_features_06-2023,features_per_home_06-2023,n_homes_10-2022,usb_homes_10-2022,total_features_10-2022,features_per_home_10-2022
state,,,,,,,,
alabama,301.0,228.0,28545.0,169.867933,175.0,122.0,17038.0,175.266917
arizona,319.0,44.0,13219.0,91.572537,343.0,72.0,16033.0,113.046444
california,1516.0,1118.0,95066.0,518.590431,1281.0,1089.0,84737.0,454.371362
colorado,218.0,4.0,13151.0,60.325688,277.0,246.0,22229.0,80.249097
delaware,148.0,8.0,8486.0,113.851708,183.0,88.0,12347.0,134.926302
florida,1974.0,165.0,91088.0,670.226843,2109.0,167.0,94061.0,593.483567
georgia,93.0,0.0,4841.0,109.092547,105.0,0.0,6316.0,119.692308
idaho,74.0,0.0,3819.0,97.327941,22.0,19.0,1654.0,75.181818
illinois,185.0,0.0,13158.0,71.124324,192.0,0.0,12808.0,66.708333


In [27]:
# % changes and save
def percentage_change(col1,col2):
    return round(((col2 - col1) / col1) * 100, 2)

In [28]:
counties['pct-change-N-homes'] = percentage_change(counties['n_homes_10-2022'],
                                                   counties['n_homes_06-2023'])
counties['pct-change-usb'] = percentage_change(counties['usb_homes_10-2022'],
                                               counties['usb_homes_06-2023'])
counties['pct-change-avg-feats-per-home'] = percentage_change(counties['features_per_home_10-2022'],
                                                              counties['features_per_home_06-2023'])

In [29]:
counties.to_csv('./analysis/usb-by-county.csv', index=False)

In [30]:
states['pct-change-N-homes'] = percentage_change(states['n_homes_10-2022'],
                                                 states['n_homes_06-2023'])    
states['pct-change-usb'] = percentage_change(states['usb_homes_10-2022'],
                                             states['usb_homes_06-2023'])
states['pct-change-avg-feats-per-home'] = percentage_change(states['features_per_home_10-2022'],
                                             states['features_per_home_06-2023'])
states.to_csv('./analysis/usb-by-state.csv')